In [ ]:
import numpy as np

# Definition of the states, accions y parameters
alpha = 0.3 # 30%

down = -50
up = 50
# State space
X = np.arange(down, up+1)

# Control space 
U = [1, 0]

# Latest time
T = 100

In [ ]:
def AVAR(q, w, alpha):
    # we recive w = w_{t+1}

    # this is something like {w(1): 20%, w(): 60%}
    eval = [(w[max(down, min(k, up))], q[k])  for k in q.keys()]
    s = sorted(eval, reverse=True)
    res = 0
    i = 0
    a = alpha
    while(alpha > 0):
        if alpha >= s[i][1]:
            res += s[i][0]*s[i][1]
            alpha -= s[i][1]
        else:
            res += s[i][0]*alpha
            alpha = 0
        i += 1
    return res/a

In [ ]:
def precal_Q(X):
    Q0 = dict()
    Q1 = dict()
    for x in X:
        Q0[x] = dict()
        Q0[x][0] = {x-2: 0.2, x-1: 0.2, x: 0.2, x+1: 0.2, x+2: 0.2}
        Q0[x][1] = {x+1: 0.4, x+2: 0.2, x+3: 0.4}
        
        Q1[x] = dict()
        Q1[x][0] = {x-1: 0.6, x: 0.2, x+1: 0.2}
        Q1[x][1] = {x-1: 0.2, x: 0.4, x+1: 0.4}
    return Q0,Q1

Q_0, Q_1 = precal_Q(X)

def Q0(x, u):
    return Q_0[x][u]

def Q1(x, u):
    return Q_1[x][u]

In [ ]:
def cost0(x, u, t):
    # x = state
    # a = action of this node
    if u == 0:
        return 0
    if u == 1:
        return 1

def cost1(x, u, t):
    # x = state
    # a = action of this node
    return np.exp(-x/20) - t 

In [ ]:
class Node:
    def __init__(self, t, id, parent = None, Q = None, cost = None):
        self.t = t # Time
        self.id = id
        self.parent = parent
        self.Q = Q
        self.cost = cost
        self.w = dict() # cost
        self.childs = []
        self.policy = dict() # One for each of the posible states 
        self.terminal = True # If it is a terminal node
        

    def new_child(self, node):
        self.childs.append(node)# Add the new child
        node.parent = self
        self.terminal = False

    def reset_w(self):
        self.w={}
    #for value iteration
    def change_w(self,new_w,gamma):
        for key in new_w.keys():
            self.w[key]=new_w[key]*(gamma**self.t) 
    #consider a node to be its own descendant
    def hasDescendant(self,nodes):
        if self in nodes:
            return True
        for child in self.childs:
            if (child.hasDescendant(nodes)):
                return True
        return False


    
    def get_w(self):
        if self.w.__len__() == 0:
            if self.terminal:
                for x in X: 
                    self.w[x] = self.cost(x, 1, self.t) # Cost of terminal nodes in this case
                    # This w is w_{t, s}
            else:
                self.calc_policy()
        
        return self.w

    def calc_policy(self):
        self.policy = {x: min(U, key=lambda u: self.cost(x, u, self.t) + sum([AVAR(self.Q(x, u), child.get_w(), alpha) for child in self.childs])) for x in X}
        self.w = {x: (self.cost(x, self.policy[x], self.t) + sum([AVAR(self.Q(x, self.policy[x]), child.get_w(), alpha) for child in self.childs])) for x in X}
        #print(f'id: {self.id}, u: {self.policy}')

    def print_tree(self, level = 0):
        print(" " * 4 * level + f'{self.id}, x_{self.t, self.id % 2}')
        for child in self.childs:
            child.print_tree(level = level+1)
            

In [ ]:
# Create tree

# ad_list is symilar to: [(0, 1), (0, 2), (1, 3), ...]
# With the relation that if we have (0,1), then 0 is parent of 1
# To use this you have to have a node list and a list of which type they are
def create_tree(nodes, ad_list):
    root = nodes[0]
    for ad in ad_list:
        nodes[ad[0]].new_child(nodes[ad[1]])
    return root


#adyacent list
ad_list = []

#The root
root = Node(0, id=0)
root.Q = Q0
root.cost = cost0

# List of nodes
nodes = [root]

# We create the adyacent list for this case
for i in range(0, T):
    new1 = Node(i+1, id=2*i+1, Q=Q1, cost=cost1)
    new2 = Node(i+1, id=2*i+2, Q=Q0, cost=cost0)
    nodes.append(new1)
    nodes.append(new2)
    ad_list.append((2*i, 2*i+1))
    ad_list.append((2*i, 2*i+2))


# Add the last node
new = Node(T+1, id = 2*T+1, Q=Q1, cost=cost1)
nodes.append(new)


l = len(ad_list)
ad_list.append((l, l+1))

print(ad_list)


In [ ]:
create_tree(nodes, ad_list)

In [ ]:
root.print_tree() 

In [ ]:
print(root.get_w())

In [ ]:



#where non_begs is all the nodes which arent in the equivalence class of root.
#begs is the set of all those equivalent to root.
#finally, the cost function itself inputted to the nodes should be based on around the samme gamma as well.


def valueIteration(root, non_begs, begs, gamma,count):
    need_to_reset = [x for x in non_begs if x.hasDescendant(nodes)]
    for i in range(0,count):
        old_w = root.get_w()
        print(old_w)
        for node in need_to_reset:
            node.reset_w()
        for node in begs:
            node.change_w(old_w,gamma)
    return root.get_w()

non_begs= [nodes[0],nodes[1]]
begs = [nodes[2]]

valueIteration(root, non_begs,begs,.9,1000)

In [ ]:
for node in nodes:
    if not node.terminal:
        print(f'id: {node.id}, u: {node.policy}')

In [ ]:
import plotly.graph_objects as go

# Crear la figura
fig = go.Figure()

for node in nodes:
    x = list(node.policy.keys())
    y = list(node.policy.values())
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=f'Time {node.t}'))

fig.update_layout(
    title='Policies by Node',
    xaxis_title='Policy',
    yaxis_title='Value',
    legend_title='Node ID',
)

fig.show()


In [ ]:
change = {}
for node in nodes:
    if node.terminal:
           print(node.id)
           continue
    prev = 1
    for x in X:
      #print(x, n.policy[x])
      if node.policy[x] != prev:
          #print(node.t, x)
          change[node.t] = x
      prev = node.policy[x]

In [ ]:
change

In [ ]:
import plotly.graph_objects as go

# Crear la figura
fig = go.Figure()

x = list(change.keys())
y = list(change.values())
fig.add_trace(go.Scatter(x=x, y=y, mode='markers'))

fig.update_layout(
    title='Policies by Node',
    xaxis_title='Time',
    yaxis_title='Value of change'
)

fig.show()
